In [1]:
# Constants

KEYWORD="data scientist"
COUNTRY = "United States"
NUM_OF_JOBS= 1000

DF_PATH_PKL = f"../data/raw/0_{NUM_OF_JOBS}_jobs_{KEYWORD}_{COUNTRY}_collected_raw_data_df.pkl"
DF_PATH_CSV = f"../data/raw/0_{NUM_OF_JOBS}_jobs_{KEYWORD}_{COUNTRY}_collected_raw_data_df.csv"

EXPORT_PATH_PKL = f"../data/raw/1_{NUM_OF_JOBS}_jobs_{KEYWORD}_{COUNTRY}_cleaned_data_df.pkl"
EXPORT_PATH_CSV = f"../data/raw/1_{NUM_OF_JOBS}_jobs_{KEYWORD}_{COUNTRY}_cleaned_data_df.csv"

FIG_DIR = "../reports/figures"

REPLACED_DICT = {
    'Founded' :             {'Nonprofit Organization' : -1, 'Company - Private' : -1,
                                'Government' : -1, 'Company - Public': -1},
    'Type of ownership' :   {'$1 to $5 billion (USD)': -1, 'Unknown / Non-Applicable': -1, '$5 to $25 million (USD)' : -1},
    'Sector' :              {'Unknown / Non-Applicable' : -1},
    'Revenue' :             {'Unknown / Non-Applicable' : -1},
    'Headquarters':         {'' : -1},
    'Size'  :               {'Unknown' : -1}   
}

# skills to investigate about
DS_SKILLS_LIST =  [ 'python','javascript','r_studio','r studio','sas','sql','postgresql','mysql',
                   'excel','tableau','power bi','power_bi','tensorflow','aws','azure',
                   'git','keras','flask','spark',
                   'docker','pytorch']

In [2]:
# Load packages
import pandas as pd
import numpy as np
from IPython.display import display

In [3]:
# Read data and print shape
raw_df = pd.read_pickle(DF_PATH_PKL)
df = raw_df.copy()
display(raw_df.shape)
# display(raw_df.head(2))
display(raw_df.columns)
display(raw_df.info())

(1000, 14)

Index(['Job Title', 'Salary Estimate', 'Job Description', 'Rating',
       'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors'],
      dtype='object')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Job Title          1000 non-null   object
 1   Salary Estimate    1000 non-null   object
 2   Job Description    1000 non-null   object
 3   Rating             1000 non-null   object
 4   Company Name       1000 non-null   object
 5   Location           1000 non-null   object
 6   Headquarters       1000 non-null   object
 7   Size               1000 non-null   object
 8   Founded            1000 non-null   object
 9   Type of ownership  1000 non-null   object
 10  Industry           1000 non-null   object
 11  Sector             1000 non-null   object
 12  Revenue            1000 non-null   object
 13  Competitors        1000 non-null   int64 
dtypes: int64(1), object(13)
memory usage: 109.5+ KB


None

In [4]:
raw_df.sample(1).iloc[0]

Job Title            HEALTHCARE DATA SCIENTIST, FCH - ENTERPRISE AN...
Salary Estimate                                                     -1
Job Description      Come join our award winning team! See What Is ...
Rating                                                             3.7
Company Name                                            Froedtert\n3.7
Location                                             Mountain View, CA
Headquarters                                     Catherine A. Jacobson
Size                                            1001 to 5000 Employees
Founded                                                           1965
Type of ownership                               Nonprofit Organization
Industry                              Health Care Services & Hospitals
Sector                                                      Healthcare
Revenue                                         $1 to $5 billion (USD)
Competitors                                                         -1
Name: 

In [5]:
# Invesigate the questionable objects columns
questionable_cols = ['Salary Estimate', 'Rating','Company Name', 'Location',
                     'Headquarters', 'Size', 'Founded','Type of ownership',
                     'Industry', 'Sector', 'Revenue', 'Competitors']

for col in questionable_cols:
    print(col)
    print(raw_df[col].unique().tolist()[:1000])
    print('--------------------------------------------')
    print()

Salary Estimate
['$87K - $121K (Glassdoor est.)', '$70K - $108K (Glassdoor est.)', -1, 'Employer Provided Salary:$67K - $120K', '$114K - $164K (Glassdoor est.)', 'Employer Provided Salary:$65K - $80K', '$99K - $145K (Glassdoor est.)', 'Employer Provided Salary:$50.00 - $55.00 Per Hour', '$84K - $129K (Glassdoor est.)', 'Employer Provided Salary:$105K - $135K', '$81K - $113K (Glassdoor est.)', '$97K - $138K (Glassdoor est.)', '$112K - $189K (Glassdoor est.)', '$76K - $110K (Glassdoor est.)', 'Employer Provided Salary:$100K - $225K', 'Employer Provided Salary:$98K - $116K', '$105K - $170K (Glassdoor est.)', 'Employer Provided Salary:$114K - $131K', '$114K - $172K (Glassdoor est.)', 'Employer Provided Salary:$150K - $160K', 'Employer Provided Salary:$78K - $150K', '$84K - $123K (Glassdoor est.)', '$91K - $122K (Glassdoor est.)', '$79K - $125K (Glassdoor est.)', 'Employer Provided Salary:$115K - $144K', '$88K - $142K (Glassdoor est.)', '$80K - $122K (Glassdoor est.)', 'Employer Provided Sa

In [6]:
################################ From Investigation ################################
# --> 'Salary Estimate'  : int
# some in ['Per Hour', 'Glassdoor est.)', 'Employer Provided Salary', '-1']
# remove all '-1' items
# in a string format, needed to remove all strings and change format into int
################################
# --> 'Rating'  : int
# in string format
# '-1' represent not collected data
# remove that have '-1' and change format to int
################################
# --> 'Company Name'  : string
# contain '\n' and the rating
# remove '\n' and the rating
################################
# --> 'Location'   : string
# contain Duplicate shortcuts ['Fort Meade, MD', 'Annapolis Junction, MD']
################################
# --> 'Headquarters'  : string
# replace '' with '-1'
# remove '-1' items
################################
# --> 'Size'  : 
# replace 'Unknown' with '-1'
# it consist of 7-classes
################################
# --> 'Founded' : int
# replace non-numeric values with '-1'
# change format to int
# remove all '-1' items
################################
# --> 'Type of ownership'   : string
# replace ['$1 to $5 billion (USD)', 'Unknown / Non-Applicable'] with '-1'
# remove all '-1' items
################################
# --> 'Industry'  : string
# remove all -1 items
################################
# --> 'Sector'  :  string 
# replace 'Unknown / Non-Applicable' with '-1'
# remove all '-1' items
################################
# --> 'Revenue'  : 
# replace 'Unknown / Non-Applicable' with '-1'
# it consist of 8-classes 
################################
# --> 'Competitors'  
# no collected data at all
# column can be removed 

In [7]:
# Tasks on data to do
##### slary parsing
##### Company name text only
##### state field
##### age of compny
##### parsing of job description (python, etc)

#####################################################
#[1] slary parsing
# df[df['Salary Estimate'] == -1]                                       # filter to view
df = df[df['Salary Estimate'] != -1]                                    # remove all '-1' items
salary = df['Salary Estimate'].apply(lambda x: x.split('(')[0])         # remove '(Glassdoor est.)' [which is glassdors estimation] in a new variable 'salary'
minus_kd = salary.apply(lambda x: x.replace('K','').replace('$',''))    # remove 'K' and '$' from last variable 'salary', in a new varaible 
min_hr = minus_kd.apply(lambda x: x.lower().replace('per hour','')\
                        .replace('employer provided salary:',''))       # replace 'per hour' & 'employer provided salary:' with '',
                                                                        # by that, the remaining values is ranged [min - max] some annuall and others on hourly

# on 'min_salary' : because there can not be min salary = 0, so if there is only one value on data , it will be the max and min slary
df['min_salary'] = min_hr.apply(lambda x: x.split('-')[0] if len(x.split('-'))==1 else x.split('-')[0] ).astype(np.float32).astype(np.int64)
df['max_salary'] = min_hr.apply(lambda x: x.split('-')[0] if len(x.split('-'))==1 else x.split('-')[1] ).astype(np.float32).astype(np.int64)
df['avg_salary'] = (df['min_salary'] + df['max_salary'])/2

# df.loc[df['Salary Estimate'].str.contains('Per Hour')]                                    # filter to view
df["hourly"] = df['Salary Estimate'].apply(lambda x: 1 if 'per hour' in x.lower() else 0)   # new column represent 'per hour' state 
df["employer_provided"] = df['Salary Estimate'].apply(lambda x: 1 if 'employer provided salary:' in x.lower() else 0)  # new column represent 'employer provided salary:' state 
 
# the state Explanation:
# hourly      -    employer_provided
#   1                     0               : The salary offered by the company per hour
#   0                     1               : The salary offered by the company per year
#   0                     0               : The salary is glosdoor estimation

In [8]:
# Test for duplicates
duplicate = df[df.duplicated()]
duplicate.shape

(546, 19)

In [9]:
# change 'Rating' format to int 
df['Rating'] = df['Rating'].astype(np.float32).astype(np.int64)
# df = df[df['Rating'] != -1]                                         # when removing all '-1' items the shape changed from (835,20) to (715,20), so Iam not suring about that step
# display(df['Rating'].dtype)

# [2] Company name text only
df['Company Name'] = df['Company Name'].str.replace('[0-9]','')\
                        .str.replace('.','').str.replace('\n','')     # removing raing numbres and '\n' 

/Users/ahmadsamir/opt/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  import sys
/Users/ahmadsamir/opt/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  


In [10]:
# [3] state field
df['job_state'] = df['Location'].apply(lambda x: x.split(',')[0]if len(x.split(',')) == 1 else x.split(',')[0])   # remove shortcuts    
df['job_state'].value_counts()
# df['some_state'] = df.apply(lambda x: 1 if x['Location'] == x['Headquarters'] else 0, axis=1)

Remote                404
Mountain View         246
O'Fallon               58
New York               30
Fort Meade             28
Cupertino              27
Annapolis Junction     25
Washington             17
Name: job_state, dtype: int64

In [11]:
# [4] replace columns values from investigation
# []
for col, replacement in REPLACED_DICT.items() :
    if col == 'Founded':
        df[col] = df[col].replace(replacement).astype(np.int64)
    else:
        df[col] = df[col].replace(replacement)
 
# [5] age of company   
# new feature 
df['Comp_Age'] = df['Founded'].apply(lambda x: x if x < 1 else 2022 - x)

In [12]:
# [6] size
df.rename(columns = {'Size':'Comp_Size'}, inplace = True)

In [13]:
# df['Location'].value_counts(normalize=True)[1]
df['Location'].value_counts(normalize=True).index

Index(['Remote', 'Mountain View, CA', 'O'Fallon, MO', 'New York, NY',
       'Fort Meade, MD', 'Cupertino, CA', 'Annapolis Junction, MD',
       'Washington, DC'],
      dtype='object')

In [14]:
# [7] parsing of job description
# investigate on skills
for skill in DS_SKILLS_LIST[:]:
    print(skill)
    temp = df['Job Description'].apply(lambda x: 1 if skill in x.lower() else 0)
    print(temp.value_counts(normalize=True,sort=True))

# Selecting skill that Presence of more than 20%
collected_skills = ['python','sql','excel','tableau','aws','git','spark']

# New feature from collected skills [yn stands for yes no]
for skill in collected_skills :
    df[f'{skill}_yn'] = df['Job Description'].apply(lambda x: 1 if skill in x.lower() else 0)


python
1    0.79521
0    0.20479
Name: Job Description, dtype: float64
javascript
0    0.997605
1    0.002395
Name: Job Description, dtype: float64
r_studio
0    1.0
Name: Job Description, dtype: float64
r studio
0    0.898204
1    0.101796
Name: Job Description, dtype: float64
sas
0    0.898204
1    0.101796
Name: Job Description, dtype: float64
sql
1    0.565269
0    0.434731
Name: Job Description, dtype: float64
postgresql
0    0.998802
1    0.001198
Name: Job Description, dtype: float64
mysql
0    0.998802
1    0.001198
Name: Job Description, dtype: float64
excel
0    0.670659
1    0.329341
Name: Job Description, dtype: float64
tableau
0    0.723353
1    0.276647
Name: Job Description, dtype: float64
power bi
0    0.948503
1    0.051497
Name: Job Description, dtype: float64
power_bi
0    1.0
Name: Job Description, dtype: float64
tensorflow
0    0.882635
1    0.117365
Name: Job Description, dtype: float64
aws
0    0.791617
1    0.208383
Name: Job Description, dtype: float64
azure
0 

### Visually verify results

In [15]:
copy_raw_df = raw_df.copy()
copy_raw_df = copy_raw_df[copy_raw_df['Salary Estimate'] != -1]         # to unify the index
i = df.sample(1).index[0]
print(copy_raw_df.iloc[i])
print('#'*100)
print(df.iloc[i])

Job Title                                        Junior Data Scientist
Salary Estimate                          $70K - $108K (Glassdoor est.)
Job Description      Job #:\nreq19774\n\nOrganization:\nWorld Bank\...
Rating                                                              -1
Company Name                                     World Bank Group\n4.2
Location                                             Mountain View, CA
Headquarters                                                        -1
Size                                                  10000+ Employees
Founded                                         Nonprofit Organization
Type of ownership                               $1 to $5 billion (USD)
Industry                                                            -1
Sector                                                              -1
Revenue                                                             -1
Competitors                                                         -1
Name: 

In [16]:
df.columns

Index(['Job Title', 'Salary Estimate', 'Job Description', 'Rating',
       'Company Name', 'Location', 'Headquarters', 'Comp_Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors',
       'min_salary', 'max_salary', 'avg_salary', 'hourly', 'employer_provided',
       'job_state', 'Comp_Age', 'python_yn', 'sql_yn', 'excel_yn',
       'tableau_yn', 'aws_yn', 'git_yn', 'spark_yn'],
      dtype='object')

In [17]:
df.to_pickle(EXPORT_PATH_PKL)
df.to_csv(EXPORT_PATH_CSV,index=False)